In [ ]:
# django project with db back end
# django-admin startproject dbproject
# cd dbproject
# python manage.py startapp dbapp
#
# edit dbproject/settings.py
#INSTALLED_APPS = [
#    ...
#    'dbapp'
#]
#
#
# DATABASES = {
#    'default': {
#        'ENGINE': 'django.db.backends.postgresql',
#        'NAME': 'sample_db_1',
#        'HOST': 'localhost',
#        'USER': 'sample_db_user_1',
#        'PASSWORD': 'password',
#        'PORT': 5432
#    }
# }
#
# create models
# edit dbapp/models.py
from django.db import models

# Create your models here.
class Book(models.Model):
    bookid = models.IntegerField()
    booktitle = models.CharField(max_length=50)
    class Meta:
        db_table = 'books'
        
# migrate models to create necessary tables
# issue the following commands from project directory
# python manage.py makemigrations
# python manage.py migrate

# create a file name 'forms.py' below dbapp
from django.forms import ModelForm
from .models import Book

class BookForm(ModelForm):
    class Meta:
        model = Book
        fields = '__all__'
        
        
# create views functions in dbapp/views.py for fetching records
from django.shortcuts import render, redirect
from django.urls import reverse
from .models import Book
from .forms import BookForm

# Create your views here.
def books(request):
    books = Book.objects.all()
    return render(request,"dbapp/books.html",{'books': books})

# create template for listing books dbapp/templates/dbapp/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>
<body>
    <h1>List of Books</h1>
    <table>
        <th>ID</th>
        <th>Title</th>
    
    {% for book in books %}
    <tr>
        <td>{{ book.bookid }}</td><td>{{ book.booktitle }}</td>
    </tr>
    {% endfor %}
    </table>
</body>
</html>

# create a file named 'urls.py' in dbapp
from django.urls import path
from . import views

urlpatterns = [
    path('book',views.books,name='books')
]


# urls.py below dbproject
from django.contrib import admin
from django.urls import path,include

urlpatterns = [
    path('dbapp/',include(('dbapp.urls','dbapp'),namespace='dbapp')),
    path('admin/', admin.site.urls),
]


# for adding record to table
# create a template addbook.html below dbapp/templates/dbapp
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>
<body>
    <h1>Add New Book</h1>
    <form action="{% url 'dbapp:addbook' }}" method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <input type="submit" value="submit">
    </form>
</body>
</html>

# update views.py below dbapp with following content
def addbooks(request):
    if request.method == 'POST':
        form = BookForm(request.POST)
        if form.is_valid():
            form.save()
            return redirect(reverse('dbapp:books'))
    else:
        form = BookForm()
        return render(request,"dbapp/addbooks.html",{'form': form})
    
# create appropriate url for view function in dbapp/urls.py (include the following line)
...some content...
path('addbook',views.addbooks,name='addbook')